### Azure AI Agent Service



In [32]:

import os
import asyncio
import yaml
from dotenv import load_dotenv

from azure.ai.projects.aio import AIProjectClient
from azure.identity.aio import DefaultAzureCredential
from semantic_kernel.agents.azure_ai import AzureAIAgent, AzureAIAgentSettings
from semantic_kernel.agents import AgentGroupChat

from semantic_kernel.contents.chat_message_content import ChatMessageContent
from semantic_kernel.contents.utils.author_role import AuthorRole

import json, re
from IPython.display import display, Markdown

# Load environment variables please TODO note I might have extra stuff in .env that needs to be cleaned out
load_dotenv()


True

In [33]:
def load_configurations():
    with open("../crewai/config/agents.yaml", "r") as f:
        agents_config = yaml.safe_load(f)
    with open("../crewai/config/tasks.yaml", "r") as f:
        tasks_config = yaml.safe_load(f)
    return agents_config, tasks_config

def build_agent_instructions(agent_config: dict, task_config: dict) -> str:
    instructions = (
        f"Role: {agent_config.get('role','')}\n"
        f"Goal: {agent_config.get('goal','')}\n"
        f"Backstory: {agent_config.get('backstory','')}\n\n"
        f"Task:\n{task_config.get('description','')}\n\n"
        f"Expected Output:\n{task_config.get('expected_output','')}\n"
    )
    return instructions


In [34]:
async def create_agent(client, agent_name: str, instructions: str) -> AzureAIAgent:
    """
    Creates an Azure AI Agent using the provided client, agent name, and instructions.
    """
    ai_agent_settings = AzureAIAgentSettings.create()
    agent_def = await client.agents.create_agent(
        model=ai_agent_settings.model_deployment_name,
        name=agent_name,
        instructions=instructions,
    )
    agent = AzureAIAgent(
        client=client,
        definition=agent_def,
    )
    return agent

async def collect_response_azure(agent: AzureAIAgent, input_text: str) -> str:
    """
    Creates a group chat with a single agent and sends the input_text.
    Returns the concatenated response text.
    """
    chat = AgentGroupChat(agents=[agent])
    await chat.add_chat_message(ChatMessageContent(role=AuthorRole.USER, content=input_text))
    response_text = ""
    async for message in chat.invoke():
        response_text += message.content
    return response_text


In [45]:
# %%
async def run_market_news_monitor(subject: str, agent_instr: str, client) -> str:
    agent = await create_agent(client, "MarketNewsMonitor", agent_instr)
    input_text = f"Subject: {subject}"
    response = await collect_response_azure(agent, input_text)
    print("======================= Market News Monitor Output ========================")
    print(response)
    return response

async def run_data_analyst(subject: str, agent_instr: str, client) -> str:
    agent = await create_agent(client, "DataAnalyst", agent_instr)
    input_text = f"Subject: {subject}"
    response = await collect_response_azure(agent, input_text)
    print("======================= Data Analyst Output ==================================")
    print(response)
    return response

async def run_content_creator(subject: str, combined_context: dict, agent_instr: str, client) -> str:
    agent = await create_agent(client, "ContentCreator", agent_instr)
    input_text = (
        f"Subject: {subject}\n\n"
        f"Market News Analysis:\n{combined_context['market_news']}\n\n"
        f"Data Analysis:\n{combined_context['data_analyst']}\n\n"
        "Based on the above, create high-quality content."
    )
    response = await collect_response_azure(agent, input_text)
    print("==================== Content Creator Output =======================")
    print(response)
    return response

async def run_quality_assurance(content: str, agent_instr: str, client) -> str:
    agent = await create_agent(client, "ContentOfficer", agent_instr)
    # input_text = (
    #     "Please review and refine the following content ensuring it is formatted in markdown "
    #     "and structured as JSON with two keys: 'article' and 'social_media_posts'. "
    #     "The 'social_media_posts' value should be a list of objects, each with 'platform' and 'content'.\n\n"
    #     f"Content:\n{content}"
    # )
    # NOTE I am adding this to get structured output TODO but I would liek to explore pydantic
    input_text = (
        "Please review and refine the following content ensuring you ONLY output one valid JSON object.\n\n"
        "Guidelines:\n"
        " - Do not include any extra text, apologies, or code fences.\n"
        " - Do not repeat the JSON block.\n"
        " - The JSON must follow this structure:\n"
        "     {\n"
        "       \"article\": { ... },\n"
        "       \"social_media_posts\": [...],\n"
        "       \"additional_sections\": { ... }\n"
        "     }\n\n"
        "Here is the content to refine:\n"
        f"{content}\n\n"
        "Again, return ONLY a single JSON object with no additional text outside the JSON.\n"
    )
    response = await collect_response_azure(agent, input_text)
    print("================== Quality Assurance Output ==================")
    print(response)
    return response


In [ ]:
# async def flow(subject: str):
#     agents_config, tasks_config = load_configurations()

#     market_news_instr = build_agent_instructions(
#         agents_config["market_news_monitor_agent"],
#         tasks_config["monitor_financial_news"]
#     )
#     data_analyst_instr = build_agent_instructions(
#         agents_config["data_analyst_agent"],
#         tasks_config["analyze_market_data"]
#     )
#     content_creator_instr = build_agent_instructions(
#         agents_config["content_creator_agent"],
#         tasks_config["create_content"]
#     )
#     quality_assurance_instr = build_agent_instructions(
#         agents_config["quality_assurance_agent"],
#         tasks_config["quality_assurance"]
#     )

#     # Create the Azure AI Agent Service client using DefaultAzureCredential. NOTE you have to do az login --tenant tenant_id for this to work 
#     ai_agent_settings = AzureAIAgentSettings.create()
#     async with (
#         DefaultAzureCredential() as creds,
#         AzureAIAgent.create_client(
#             credential=creds,
#             conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
#         ) as client
#     # async with (
#     # DefaultAzureCredential() as creds,
#     #     AIProjectClient(
#     #         credential=creds,
#     #         endpoint=ai_agent_settings.endpoint,
#     #         subscription_id=ai_agent_settings.subscription_id,
#     #         resource_group_name=ai_agent_settings.resource_group_name,
#     #         project_name=ai_agent_settings.project_name
#     #     ) as client,
#     ):
#         # Run the first two agents in parallel (Market News Monitor and Data Analyst)
#         market_news_future = run_market_news_monitor(subject, market_news_instr, client)
#         data_analyst_future = run_data_analyst(subject, data_analyst_instr, client)
#         market_news_result, data_analyst_result = await asyncio.gather(market_news_future, data_analyst_future)

#         # Combine the outputs from the first two agents
#         combined_context = {
#             "market_news": market_news_result,
#             "data_analyst": data_analyst_result
#         }

#         # Run Content Creator agent with the combined context
#         content_output = await run_content_creator(subject, combined_context, content_creator_instr, client)

#         # Run Quality Assurance agent on the content output
#         final_output = await run_quality_assurance(content_output, quality_assurance_instr, client)

#         print("\n======================== FINAL STRUCTURED OUTPUT =========================")
#         print(final_output)
#         return final_output


In [ ]:
async def flow(subject: str):
    
  
    agents_config, tasks_config = load_configurations()

    market_news_instr = build_agent_instructions(
        agents_config["market_news_monitor_agent"],
        tasks_config["monitor_financial_news"]
    )
    data_analyst_instr = build_agent_instructions(
        agents_config["data_analyst_agent"],
        tasks_config["analyze_market_data"]
    )
    content_creator_instr = build_agent_instructions(
        agents_config["content_creator_agent"],
        tasks_config["create_content"]
    )
    quality_assurance_instr = build_agent_instructions(
        agents_config["quality_assurance_agent"],
        tasks_config["quality_assurance"]
    )

    ai_agent_settings = AzureAIAgentSettings.create()
    async with (
        DefaultAzureCredential() as creds,
        AzureAIAgent.create_client(
            credential=creds,
            conn_str=ai_agent_settings.project_connection_string.get_secret_value(),
        ) as client
    ):
        # Run the first two agents in parallel
        market_news_future = run_market_news_monitor(subject, market_news_instr, client)
        data_analyst_future = run_data_analyst(subject, data_analyst_instr, client)
        market_news_result, data_analyst_result = await asyncio.gather(
            market_news_future, data_analyst_future
        )

        # Combine the outputs
        combined_context = {
            "market_news": market_news_result,
            "data_analyst": data_analyst_result
        }

        # Run Content Creator agent with the combined context
        content_output = await run_content_creator(subject, combined_context, content_creator_instr, client)

        # Run Quality Assurance to get strictly valid JSON
        final_output_str = await run_quality_assurance(content_output, quality_assurance_instr, client)

    # I am getting error here TODO fix this somehow using strucutred out
    final_output_json = None
    try:
        final_output_json = json.loads(final_output_str)
    except json.JSONDecodeError as e:
        print("\nERROR: The QA output was not valid JSON.")
        print("JSONDecodeError:", e)

    print("\n======================== FINAL STRUCTURED OUTPUT =========================")
    print(final_output_json)

    return final_output_json

In [47]:
# %%
final_output = await flow("Inflation in the US and the impact on the stock market in 2024")


======================= Data Analyst Output ==================================
## Comprehensive Analysis Report: Inflation in the US and its Impact on the Stock Market in 2024

### Executive Summary
This report analyzes the potential impact of inflation in the United States on the stock market in 2024. Utilizing advanced data analytics tools, we examine historical trends, market data, and economic indicators to provide actionable insights. Key patterns, opportunities, and risks are identified, which can guide content creation strategies aimed at investors and financial analysts.

### Key Market Trends

1. **Historical Inflation Trends and Correlations:**
   - **Data Analysis:** Studying data from the past two decades reveals a cyclical relationship between inflation rates and stock market performance. Moderate inflation typically correlates with positive stock market returns, while high inflation can erode earnings and compress equity valuations.
   - **Visualizations:** Historical inf

In [40]:
type(final_output)

NoneType

In [ ]:
from IPython.display import display, Markdown

blog_content = final_output["article"]["content"]

# Display the blog content
display(Markdown(blog_content))


TypeError: 'NoneType' object is not subscriptable

In [ ]:
from IPython.display import display, Markdown

social_posts = final_output_json.get("social_media_posts", [])

# Loop over each social media post and display it neatly
for idx, post in enumerate(social_posts, start=1):
    platform = post.get("platform", "Unknown platform")
    content = post.get("content", "No content")
    
    # Format as Markdown
    display(Markdown(f"### Post {idx}\n**Platform:** {platform}\n\n**Content:**\n{content}"))
    print("---")
